In [1]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset
import numpy as np
import json

In [2]:
pwd

'/home/al2644/research/codebase/reasoning/perturb-r/notebook'

In [4]:
cd /home/al2644/research/codebase/reasoning/perturb-r

/home/al2644/research/codebase/reasoning/perturb-r


In [5]:
from utils import chunk_r 

# Reasoning Results

In [7]:
root = "./results/deepmath_7to9/benchmark"
for fname in os.listdir(root):
    if 'correct' not in fname:
        print(fname)
        df = pd.read_pickle(os.path.join(root, fname))
        print(f"Accuracy: {df['correct'].mean()}")

R1-Distill-Qwen-1.5B.pickle
Accuracy: 0.4706666666666667
R1-Distill-Qwen-7B.pickle
Accuracy: 0.6553333333333333
Qwen3-1.7B.pickle
Accuracy: 0.56
Qwen3-4B.pickle
Accuracy: 0.7246666666666667
Qwen3-1.7B_nothinking.pickle
Accuracy: 0.24133333333333334
Qwen3-4B_nothinking.pickle
Accuracy: 0.3293333333333333
Qwen3-8B.pickle
Accuracy: 0.7453333333333333
Qwen3-8B_nothinking.pickle
Accuracy: 0.3426666666666667
Qwen3-30B-A3B.pickle
Accuracy: 0.7946666666666666
Qwen3-30B-A3B_nothinking.pickle
Accuracy: 0.36933333333333335


In [22]:
think_df = pd.read_pickle(os.path.join(root, 'Qwen3-1.7B.pickle'))
nothink_df = pd.read_pickle(os.path.join(root, 'Qwen3-1.7B_nothinking.pickle'))

In [52]:
problems = think_df.groupby("problem")[["correct"]].all()
problems[problems["correct"]]

,correct
problem,
"Calculate $248^{156,454,638} \mod 250$ without using a computer.",True
Calculate \( \lim\limits_{n\to\infty}\lim\limits_{x\to\ 0}I^{(n)}_{-\frac{1}{2}+n}(\sqrt{x}) \) using the power series representation of the Bessel function.,True
Calculate the expectation \( E[W_s e^{W_s - W_t}] \) for a Brownian process \( W_s \) where \( 0 \leq s < t \) and \( W_0 = 0 \).,True
"Calculate the expected time until a bank balance, modeled by a Brownian motion with positive drift, first becomes negative.",True
Calculate the limit \( \lim_{n\to\infty}\int_{0}^{1}\frac{n^{1/3}}{1+n\cos(x)}dx \).,True
...,...
"Suppose $f:E\rightarrow E'$ is a linear embedding between two locally convex topological vector spaces. Given an open, convex subset $A\subseteq E$ with $0\in A$, determine if there exists an open, convex set $A'\subseteq E'$ such that $f(A)=f(E)\cap A'$.",True
"Suppose $f\colon [0,\infty)\to\mathbb{R}$ is continuously differentiable and both $f$ and $f'$ are Lebesgue integrable on $[0,\infty)$. Prove that $\lim_{x\to\infty}f(x)=0$. Provide a detailed explanation of your reasoning.",True
Suppose \(\{a_n\}_{n\geq 1}\) is a sequence of positive real numbers such that the series \( \sum_{k=1}^{\infty} a_k \) converges and \( \lim_{n \to \infty} \frac{a_{n+1}}{a_n}=1 \). Determine whether it is always true that \( \lim_{n \to \infty} na_n = 0 \). Provide a proof or a counterexample.,True


In [27]:
think_problems = think_df.groupby("problem")[["correct"]].sum()
think_problems = set(think_problems[think_problems["correct"] > 2].index)

In [32]:
len(think_problems)

374

In [40]:
nothink_problems = nothink_df.groupby("problem")[["correct"]].sum()
nothink_problems = set(nothink_problems[nothink_problems["correct"] > 0].index)

In [41]:
len(nothink_problems)

175

In [39]:
len(think_problems.difference(nothink_problems))

218

# Extract 1st Answer Attempt

In [190]:
idx = 20
example = df.iloc[idx]
problem, answer, chunks, index = example["problem"], example["answer"], example["chunks"], example["1st_answer_chunk_index"]

In [185]:
stream = ""
for idx, chunk in enumerate(chunks):
    stream += f"\n>>>>>CHUNK{idx} START\n" + chunk + f"\n>>>>>CHUNK{idx} END\n"

In [186]:
instruct_prompt = """<instruction>
You are given a sequence of reasoning steps, divided into chunks, showing how a model solves a problem. Your task is to precisely identify the first potential answer that model has reached. The first candidate does NOT NEED to be correct. But, the model should either explicitly mention that it can be the answer or hint that it thinks that it can be the answer before any checking or verification. The first answer candidate does NOT need to be confirmed, verified, or boxed. 
    
You should scan through the chunks from the start, ensuring that the candidate that you detect is indeed the earliest attempt:
1.    Skim over earlier chunks that merely state the problem, set up notation, or provide just derivation.
2.    Dismiss later chunks that just verify or check the first answer candidate.
3.    The first answer candidate may be later checked or even changed. But you SHOULD NOT report later versions.
4.    After you locate and identify the first answer candidate, you MUST verify that it is the very first answer candidate by scanning backward carefully to ensure that earlier chunks do not any potential answer attempt. If mentioned before, update your answer. 

You should first spend time verbalizing your searching and verification process within <search></search> tags. And once identified, you should report the chunk number in <chunk_number></chunk_number> tags and put the candidate in <first_candidate></first_candidate> tags. Follow this response format:
<search>YOU_CAN_SEARCH_HERE</search>
<chunk_number>NUMBER</chunk_number>
<first_candidate>FIRST_CANDIDATE</first_candidate>

Regardless of its correctness, FIRST_CANDIDATE should be formatted like the final solution ({answer}) so that I can easily verify it matches the final answer.
</instruction>

problem: {problem}
reasoning trace:
""".format(answer = answer, problem = problem)

In [187]:
prompt = instruct_prompt + stream

In [188]:
print(prompt)

<instruction>
You are given a sequence of reasoning steps, divided into chunks, showing how a model solves a problem. Your task is to precisely identify the first potential answer that model has reached. The first candidate does NOT NEED to be correct. But, the model has to either explicitly mention that it can be the answer or hint that it thinks that it can be the answer before any checking or verification. The first answer candidate does NOT need to be confirmed, verified, or boxed. 
    
You should scan througth the chunks from the start, ensuring that the candidate that you detect is indeed the earliest attempt:
1.    Skim over earlier chunks that merely state the problem, set up notation, or provide just derivation.
2.    Dismiss later chunks that just verify or check the first answer candidate.
3.    The first answer candidate may be later checked or even changed. But you SHOULD NOT report later versions.
4.    After you locate and identify the first answer candidate, you MUST v